<a href="https://colab.research.google.com/github/bishair/Pirna/blob/main/CDT125(3G29).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import matplotlib.pyplot as plt
from IPython.display import display
import joblib


In [2]:
uploaded_file = files.upload()

Saving trainingdata.csv to trainingdata.csv


In [36]:
# Load the data
data = pd.read_csv('trainingdata.csv')

In [37]:
# Filter the data for waterpoint_id 3
gw = data[data['waterpoints_id'] == 3]
#print(gw)

In [38]:
gw.dropna(inplace=True)
#print(gw)

<ipython-input-38-0527c3859693>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gw.dropna(inplace=True)


In [41]:
#gw = gw.drop(['id', 'waterpoints_id', 'waterpoints_name'], axis=1)
#gw.rename(columns={'water_level': 'G29', 'date_time': 'Date'}, inplace=True)
print(gw)

                   Date      G29
2652   10/31/2023 13:00  109.594
2653   10/31/2023 14:00  109.593
2654   10/31/2023 15:00  109.592
2655   10/31/2023 16:00  109.591
2656   10/31/2023 17:00  109.589
...                 ...      ...
19365   6/30/2023 19:00  109.392
19366   6/30/2023 20:00  109.392
19367   6/30/2023 21:00  109.390
19368   6/30/2023 22:00  109.389
19369   6/30/2023 23:00  109.387

[6045 rows x 2 columns]


In [42]:
# Filter the data for waterpoint_id 6
river = data[data['waterpoints_id'] == 6]
#print(river)

In [43]:
river.dropna(inplace=True)
#print(river)

<ipython-input-43-04faa066f5bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  river.dropna(inplace=True)


In [44]:
river = river.drop(['id', 'waterpoints_id', 'waterpoints_name'], axis=1)
river.rename(columns={'water_level': 'river', 'date_time': 'Date'}, inplace=True)
print(river)

                  Date   river
19370   1/20/2023 9:00  116.75
19371  1/20/2023 10:00  116.75
19372  1/20/2023 11:00  116.76
19373  1/20/2023 12:00  116.76
19374  1/20/2023 13:00  116.74
...                ...     ...
29287  1/17/2023 19:00  116.86
29288  1/17/2023 20:00  116.86
29289  1/17/2023 21:00  116.86
29290  1/17/2023 22:00  116.86
29291  1/17/2023 23:00  116.86

[9906 rows x 2 columns]


In [49]:
gw['Date'] = pd.to_datetime(gw['Date'], format='%m/%d/%Y %H:%M')
river['Date'] = pd.to_datetime(river['Date'], format='%m/%d/%Y %H:%M')

In [53]:
# Merge the two datasets on the 'Date' column
merged_data = pd.merge_asof(gw.sort_values('Date'), river.sort_values('Date'), on='Date', direction='nearest')
# Set 'Date' as the index
merged_data.set_index('Date', inplace=True)
print(merged_data)

                         G29    river
Date                                 
2022-12-19 06:00:00  109.755  115.700
2022-12-19 07:00:00  109.753  115.690
2022-12-19 08:00:00  109.750  115.690
2022-12-19 09:00:00  109.747  115.690
2022-12-19 10:00:00  109.745  115.680
...                      ...      ...
2023-12-28 08:00:00  112.605  120.415
2023-12-28 09:00:00  112.613  120.425
2023-12-28 10:00:00  112.621  120.430
2023-12-28 11:00:00  112.629  120.440
2023-12-28 12:00:00  112.637  120.440

[6045 rows x 2 columns]


In [56]:
n_steps = 24

# Split the original data first before scaling
train_size = int(len(merged_data) * 0.7)
resampled_data_train = merged_data[:train_size]
resampled_data_test = merged_data[train_size:]

# Apply MinMaxScaler only to the training data
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(resampled_data_train)
test_scaled = scaler.transform(resampled_data_test)

joblib.dump(scaler, 'scaler.gz')


# Function to create sequences
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

# Create sequences for training and testing sets
X_train, y_train = create_sequences(train_scaled, n_steps)
X_test, y_test = create_sequences(test_scaled, n_steps)

# Print shapes of X_train, y_train, X_test, y_test for verification
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (4207, 24, 2)
y_train shape: (4207,)
X_test shape: (1790, 24, 2)
y_test shape: (1790,)


In [58]:
#print(X_train, X_train.shape)

In [60]:
#print(X_test, X_test.shape)

In [62]:
print(y_train, y_train.shape)
print(y_test, y_test.shape)

[0.15641991 0.15600167 0.1551652  ... 0.09242995 0.09201171 0.09326642] (4207,)
[0.09619406 0.0966123  0.0966123  ... 1.37557507 1.37892095 1.38226683] (1790,)


In [63]:
input_shape=(X_train.shape[1], X_train.shape[2])

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l1_l2

model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=input_shape,   #input shape => (timesteps, features).  X => (samples, timesteps, features)
         dropout=0.2, recurrent_dropout=0.2,
         kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
    Dropout(0.2),
    LSTM(units=50, dropout=0.2, recurrent_dropout=0.2),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
66/66 [==============================] - 13s 82ms/step - loss: 0.3024
Epoch 2/50
66/66 [==============================] - 5s 69ms/step - loss: 0.1115
Epoch 3/50
66/66 [==============================] - 7s 103ms/step - loss: 0.0288
Epoch 4/50
66/66 [==============================] - 5s 70ms/step - loss: 0.0133
Epoch 5/50
66/66 [==============================] - 5s 69ms/step - loss: 0.0104
Epoch 6/50
66/66 [==============================] - 6s 95ms/step - loss: 0.0090
Epoch 7/50
66/66 [==============================] - 5s 69ms/step - loss: 0.0077
Epoch 8/50
66/66 [==============================] - 6s 85ms/step - loss: 0.0078
Epoch 9/50
66/66 [==============================] - 5s 77ms/step - loss: 0.0078
Epoch 10/50
66/66 [==============================] - 5s 69ms/step - loss: 0.0073
Epoch 11/50
66/66 [==============================] - 6s 95ms/step - loss: 0.0068
Epoch 12/50
66/66 [==============================] - 5s 70ms/step - loss: 0.0072
Epoch 13/50
66/66 [================

In [65]:
mse = model.evaluate(X_test, y_test)
print(mse)


56/56 [==============================] - 3s 15ms/step - loss: 0.0019
0.0019068702822551131


In [66]:
model.save('lstmmodel.h5')
model = load_model('lstmmodel.h5')
scaler = joblib.load('scaler.gz')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#Preprocess the new data using similar steps used before for creating a merged dataframe to combine river and gw values
#Discard the index points used for training the model, this value will be fixed as the model has already been trained on this data



In [ ]:
actual_values = []
predicted_values = []